# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
geoapify_key

'f25e0b7f404546b586ff186c08079b78'

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,saint-pierre,-21.3393,55.4781,21.98,68,0,2.57,RE,1696302137
1,1,ha'il,27.5219,41.6907,24.01,35,0,0.00,SA,1696302137
2,2,waitangi,-43.9535,-176.5597,8.23,92,100,7.60,NZ,1696302127
3,3,vilyuchinsk,52.9306,158.4028,9.15,85,100,3.07,RU,1696302137
4,4,invercargill,-46.4000,168.3500,16.68,49,6,2.68,NZ,1696302137


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_1 = city_data_df.hvplot.points("Lng", "Lat", geo=True, tiles ="OSM", frame_width=700, frame_height=500, size="Humidity", color="City", alpha = 0.5)


# Display the map
# YOUR CODE HERE
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
criteria = (city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21) & (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)

city_data_filtered_df = city_data_df.loc[criteria].copy()

# Drop any rows with null values
city_data_filtered_df.dropna(inplace=True)

# Display sample data
city_data_filtered_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,saint-pierre,-21.3393,55.4781,21.98,68,0,2.57,RE,1696302137
1,1,ha'il,27.5219,41.6907,24.01,35,0,0.00,SA,1696302137
74,74,conde,-7.2597,-34.9075,22.70,94,0,2.06,BR,1696302147
123,123,ghat,24.9647,10.1728,25.30,38,0,1.32,LY,1696302145
140,140,afif,23.9065,42.9172,24.13,20,0,3.54,SA,1696302155
163,163,laguna,38.4210,-121.4238,22.40,42,0,2.57,US,1696302158
246,246,gebeit,21.0667,36.3167,24.72,55,0,2.41,SD,1696302176
254,254,al bardiyah,31.7561,25.0865,22.75,64,0,4.40,LY,1696302185
267,267,as sulayyil,20.4607,45.5779,22.14,17,0,0.88,SA,1696302188
270,270,hamilton,39.1834,-84.5333,21.89,80,0,0.00,US,1696302188


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_filtered_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head(20)

,City,Country,Lat,Lng,Humidity,Hotel Name
0,saint-pierre,RE,-21.3393,55.4781,68,
1,ha'il,SA,27.5219,41.6907,35,
74,conde,BR,-7.2597,-34.9075,94,
123,ghat,LY,24.9647,10.1728,38,
140,afif,SA,23.9065,42.9172,20,
163,laguna,US,38.4210,-121.4238,42,
246,gebeit,SD,21.0667,36.3167,55,
254,al bardiyah,LY,31.7561,25.0865,64,
267,as sulayyil,SA,20.4607,45.5779,17,
270,hamilton,US,39.1834,-84.5333,80,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "categories": "accomodation.hotel",
    "radius":radius,
    "limit": 1,
    "sort": "distance"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{radius}@{lng},{lat}"
    params["bias"] = f"circle:{radius}@{lng},{lat}"
            
    # Set base URL
    base_url = f"https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
saint-pierre - nearest hotel: No hotel found
ha'il - nearest hotel: No hotel found
conde - nearest hotel: No hotel found
ghat - nearest hotel: No hotel found
afif - nearest hotel: No hotel found
laguna - nearest hotel: No hotel found
gebeit - nearest hotel: No hotel found
al bardiyah - nearest hotel: No hotel found
as sulayyil - nearest hotel: No hotel found
hamilton - nearest hotel: No hotel found
two rivers - nearest hotel: No hotel found
holualoa - nearest hotel: No hotel found
el quebrachal - nearest hotel: No hotel found
khash - nearest hotel: No hotel found
hanceville - nearest hotel: No hotel found
yuma - nearest hotel: No hotel found
iowa city - nearest hotel: No hotel found
lampasas - nearest hotel: No hotel found
beacon square - nearest hotel: No hotel found
remire-montjoly - nearest hotel: No hotel found
bay harbor - nearest hotel: No hotel found
at taraf - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
0,saint-pierre,RE,-21.3393,55.4781,68,No hotel found
1,ha'il,SA,27.5219,41.6907,35,No hotel found
74,conde,BR,-7.2597,-34.9075,94,No hotel found
123,ghat,LY,24.9647,10.1728,38,No hotel found
140,afif,SA,23.9065,42.9172,20,No hotel found
163,laguna,US,38.4210,-121.4238,42,No hotel found
246,gebeit,SD,21.0667,36.3167,55,No hotel found
254,al bardiyah,LY,31.7561,25.0865,64,No hotel found
267,as sulayyil,SA,20.4607,45.5779,17,No hotel found
270,hamilton,US,39.1834,-84.5333,80,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot

map_plot_2 = hotel_df.hvplot.points( "Lng", "Lat", geo = True, tiles = "OSM", frame_width = 700, frame_height = 400, size = "Humidity", color = "City", alpha = 0.5, hover_cols=["Hotel Name", "Country"])

#points = gv.Points(hotel_df, kdims=["Lng", "Lat"], vdim=["City", "Country", "Hotel Name"])

#hover = gv.operation.datashader.bundle.shade(points, cmap=["steelblue"], alpha=255),opts(tools=['hover'], tooltips=[('City','@City'), ('Country', '@Country'), ('Hotel Name', '@Hotel Name')])

#map_plot_2 = points.hvplot(geo=True, tiles="OSM", frame_width=700, frame_height=500, size="Humidity",color="City",alpha=0.5).opts(tools=['hover'], hover=hover)

#map_plot_2 = hotel_df.hvplot.points("Lng", "Lat", geo=True, tiles ="OSM", frame_width=700, frame_height=500, size="Humidity", color="City", alpha = 0.5)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)

In [9]:
print(base_url)

https://api.geoapify.com/v2/places
